In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from signals import *
from frequencyestimator import *
from scipy.optimize import basinhopping, minimize
from tqdm.auto import tqdm

sns.set_style("whitegrid")
sns.despine(left=True, bottom=True)
sns.set_context("poster", font_scale = .45, rc={"grid.linewidth": 0.8})

/Users/farrokhlabib/Documents/github/csAE/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<Figure size 640x480 with 0 Axes>

In [2]:
P0 = lambda n, theta: np.cos((2*n+1)*theta)**2
P1 = lambda n, theta: np.sin((2*n+1)*theta)**2

def estimate_signal(depths, n_samples, theta):
        signals = np.zeros(len(depths), dtype = np.complex128)
        measurements = np.zeros(len(depths))
        for i,n in enumerate(depths):
            # Get the exact measuremnt probabilities
            p0 = P0(n, theta)
            p1 = P1(n, theta)

            p0x = P0x(n,theta)
            p1x = P1x(n,theta)
            
            p0_estimate = np.random.binomial(n_samples[i], p0)/n_samples[i]
            p1_estimate = 1 - p0_estimate

            p0x_estimate = np.random.binomial(n_samples[i], p0x)/n_samples[i]
            p1x_estimate = 1.0 - p0x_estimate
            measurements[i] = p0_estimate
            
            # Estimate theta
            theta_estimated = np.arctan2(p0x_estimate - p1x_estimate, p0_estimate - p1_estimate)

            # Compute f(n) - Eq. 3
            fi_estimate = np.exp(1.0j*theta_estimated)
            signals[i] = fi_estimate
         
        return signals, measurements

from scipy.stats import binom

def apply_correction(ula_signal, measurements, theta_est, theta):
    p_exact = np.cos((2*ula_signal.depths+1)*(theta))**2
    p_neg = np.cos((2*ula_signal.depths+1)*(-theta))**2
    p_o2 = np.cos((2 * ula_signal.depths + 1) * (theta_est/2.0)) ** 2
    p_o4 = np.cos((2 * ula_signal.depths + 1) * (theta_est/4.0)) ** 2
    p_same = np.cos((2*ula_signal.depths+1)*(theta_est))**2
    p_s2 = np.cos((2 * ula_signal.depths + 1) * (np.pi/2-theta_est)) ** 2
    p_s4 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 4 - theta_est)) ** 2
    p_s2_o2 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 2 - theta_est/2)) ** 2
    p_s4_o2 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 4 - theta_est/2)) ** 2

    l_exact = np.sum(
        np.log([1e-75 + binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk],
                                  p_exact[kk]) for kk in
                range(len(ula_signal.n_samples))]))
    l_neg = np.sum(
        np.log([1e-75 + binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk],
                                  p_neg[kk]) for kk in
                range(len(ula_signal.n_samples))]))
    l_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk]*measurements[kk], ula_signal.n_samples[kk], p_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_o4 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_o4[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_same = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_same[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s4 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s4[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s2_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s2_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s4_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s4_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    
    which_correction = np.argmax([l_same, l_s2, l_s4, l_o2, l_o4, l_s2_o2, l_s4_o2, l_neg])
    if which_correction == 1:
        theta_est = np.pi/2.0 - theta_est
    elif which_correction == 2:
        theta_est = np.pi/4.0 - theta_est
    elif which_correction == 3:
        theta_est = theta_est/2
    elif which_correction == 4:
        theta_est = theta_est/4
    elif which_correction == 5:
        theta_est = np.pi / 2.0 - 0.5 * theta_est
    elif which_correction == 6:
        theta_est = np.pi / 4.0 - 0.5 * theta_est
    # elif which_correction == 7:
    #     theta_est = -theta_est

    # print(f'FINAL ANGLE FOUND: {theta_est, theta}')

    return theta_est

In [3]:
import numpy as np
import itertools
from typing import List

def generate_all_sign_variations(signs_array: np.ndarray) -> np.ndarray:
    """
    Generate all possible sign variations by changing signs at all possible pairs of positions.
    
    Parameters:
    -----------
    signs_array : numpy.ndarray
        Original array of signs (typically containing 1 or -1)
    
    Returns:
    --------
    numpy.ndarray
        Array of all possible sign variation arrays
    """
    # Get all possible unique pairs of positions
    n = len(signs_array)
    position_pairs = list(itertools.combinations(range(n), 2))
    
    # Will store all variations
    all_variations = []
    
    # Iterate through all possible position pairs
    for pos1, pos2 in position_pairs:
        # Generate variations for this pair of positions
        pair_variations = generate_pair_variations(signs_array, pos1, pos2)
        all_variations.extend(pair_variations)
    
    return all_variations

def generate_adjacent_sign_variations(signs_array: np.ndarray) -> np.ndarray:
    """
    Generate sign variations by sliding a two-position window across the array.
    
    Parameters:
    -----------
    signs_array : numpy.ndarray
        Original array of signs (typically containing 1 or -1)
    
    Returns:
    --------
    numpy.ndarray
        Array of all possible sign variation arrays
    """
    # Total length of the array
    n = len(signs_array)
    
    # Will store all variations
    all_variations = []
    
    # Iterate through adjacent position pairs 
    # (0,1), (1,2), (2,3), ... until the second-to-last pair
    for pos1 in range(1, n - 1):
        pos2 = pos1 + 1
        
        # Generate variations for this pair of adjacent positions
        pair_variations = generate_pair_variations(signs_array, pos1, pos2)
        all_variations.extend(pair_variations)
    
    return all_variations

def generate_pair_variations(signs_array: np.ndarray, pos1: int, pos2: int) -> List[np.ndarray]:
    """
    Generate sign variations for a specific pair of positions.
    
    Parameters:
    -----------
    signs_array : numpy.ndarray
        Original array of signs
    pos1 : int
        First position to modify
    pos2 : int
        Second position to modify
    
    Returns:
    --------
    List[numpy.ndarray]
        List of sign variation arrays
    """
    # Validate input positions
    if pos1 < 0 or pos2 < 0 or pos1 >= len(signs_array) or pos2 >= len(signs_array):
        raise ValueError("Positions must be within the array bounds")
    
    # Generate all possible sign combinations for the two positions
    sign_combinations = list(itertools.product([-1, 1], repeat=2))
    
    # Create variations
    variations = []
    for combo in sign_combinations:
        # Create a copy of the original array
        variation = signs_array.copy()
        
        # Modify the two specified positions
        variation[pos1] *= combo[0]
        variation[pos2] *= combo[1]
        
        variations.append(tuple(variation))
    
    return variations


# Example array
signs = np.array([1]*12)

# Generate variations for all position pairs
all_variations = generate_adjacent_sign_variations(signs)

print("Original array:", signs)
print(f"\nTotal variations: {len(all_variations)}")
print(f'4*len(signs)^2: {4*len(signs)**2}')
print(f'2^len(signs): {2**len(signs)}')

print("\nFirst few variations:")
for var in all_variations[:10]:  # Print first 10 variations
    print(var)

Original array: [1 1 1 1 1 1 1 1 1 1 1 1]

Total variations: 40
4*len(signs)^2: 576
2^len(signs): 4096

First few variations:
(1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
(1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
(1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
(1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1)
(1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
(1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1)
(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
(1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1)
(1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1)


## Define the objective function to minimize

In [4]:
def objective_function(lp, cos_signal, abs_sin, ula_signal, esprit):   
    signal = cos_signal + 1.0j * lp * abs_sin
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    _, _ = esprit.estimate_theta_toeplitz(R)
    eigs = np.abs(esprit.eigs)[:2]
    obj = eigs[1] - eigs[0]

    return obj

## What is the distribution of the signs? 

In [5]:
narray = [2,2,2,2,2,2,2,3]

# avals = np.linspace(0.1, 0.9, 8)
avals = [0.1*i for i in range(1, 10)]

ula_signal = TwoqULASignal(M=narray, C=5)
esprit = ESPIRIT()

num_mc = 1000

sign_distributions = {}

for a in avals:
    theta = np.arcsin(a)
    distr = {}
    for i in range(num_mc):
        signal, _ = estimate_signal(ula_signal.depths, ula_signal.n_samples, theta)
        # print(measurements)
        signs = tuple(np.sign(np.imag(signal)))
        # print(signs)

        distr[signs] = distr.get(signs, 0.0) + 1/num_mc
    sign_distributions[str(a)] = distr



In [6]:
2**len(ula_signal.depths), len(sign_distributions['0.1'])

(1024, 19)

In [7]:
all_signs = set()
for a in avals:
    distr = sign_distributions[str(a)]
    for sign in distr.keys():
        all_signs.add(sign)
len(all_signs)

123

In [8]:
len(all_signs)/2**len(ula_signal.depths)

0.1201171875

In [9]:
narray = [2,2,2,2,2,3]

def get_heavy_signs(narray, steps):

    thetas = np.linspace(np.arcsin(0.09), np.arcsin(0.91), steps)
    avals = np.sin(thetas)

    ula_signal = TwoqULASignal(M=narray, C=5)

    num_mc = 1000

    sign_distributions = {}

    for a,theta in zip(avals,thetas):
        # theta = np.arcsin(a)
        distr = {}
        for i in range(num_mc):
            signal, _ = estimate_signal(ula_signal.depths, ula_signal.n_samples, theta)
                # print(measurements)
            signs = tuple(np.sign(np.imag(signal)))
            # print(signs)

            distr[signs] = distr.get(signs, 0.0) + 1/num_mc
        sign_distributions[str(a)] = distr

    def get_signs(sign_distribution):
        # Sort the dictionary by values in descending order
        sorted_data = dict(sorted(sign_distribution.items(), key=lambda item: item[1], reverse=True))

        # Create a new dictionary with cumulative sum close to 0.68
        cumulative_dict = {}
        cumulative_sum = 0.0

        for key, value in sorted_data.items():
            cumulative_dict[key] = value
            if cumulative_sum + value > 0.9:
                break
            cumulative_sum += value

        # Output the result
        return cumulative_dict
    
    ret_dict = {}
    for a in avals:
        distr = get_signs(sign_distributions[str(a)])
        ret_dict[str(a)] = distr
 
    # Normalize the values
    normalized_data = {}
    for key, sub_dict in ret_dict.items():
        total_sum = sum(sub_dict.values())
        normalized_data[key] = {k: v / total_sum for k, v in sub_dict.items()}

    return normalized_data

normalized_data = get_heavy_signs(narray, 10)


In [10]:
import ast
# Taking a random sample based on the normalized probabilities
def sample_from_normalized(data, sample_size=1):
    keys = list(map(str, list(data.keys())))
    probabilities = list(data.values())
    sampled_keys = np.random.choice(a=keys, size=sample_size, p=probabilities)
    return [ast.literal_eval(t) for t in sampled_keys]
avals = list(normalized_data.keys())
# Get one random sample
sampled = sample_from_normalized(normalized_data[avals[0]], sample_size=3)
print("Sampled keys:",sampled)

Sampled keys: [(1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0), (1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0), (1.0, 1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0)]


In [11]:
aa_milne_arr = ['pooh', 'rabbit', 'piglet', 'Christopher']
np.random.choice(aa_milne_arr, 5, p=[0.5, 0.1, 0.1, 0.3])

array(['pooh', 'rabbit', 'rabbit', 'pooh', 'pooh'], dtype='<U11')

## Use the distribution of signs so we don't have to try all possible signs

In [12]:
# np.random.seed(42)

a = 0.2
print(a)
theta = np.arcsin(a)

narray = [2,2,2,2,2,2,2,2]
heavy_signs = get_heavy_signs(narray, len(narray)**2)

ula_signal = TwoqULASignal(M=narray, C=5)
esprit = ESPIRIT()

def csae_with_local_minimization(theta, ula_signal, esprit, heavy_signs, sample=False, correction=False, optimize=False, method='nelder-mead', maxiter=10, disp=False):

    depths = ula_signal.depths
    n_samples = ula_signal.n_samples
    csignal, measurements = estimate_signal(depths, n_samples, theta)

    cos_signal = np.real(csignal)

    correct_signs = np.sign(np.imag(csignal))
    if disp:
        print(f'correct signs: {correct_signs}')
    abs_sin = np.abs(np.imag(csignal))

    obj = 1.0
    if sample:
        avals = list(heavy_signs.keys())
        signs_to_try = {}
        for a in avals:
            signs_to_try[a] = list(set(sample_from_normalized(heavy_signs[a], sample_size=2)))

        if optimize:
            # rough estimate where the amplitude is
            a0 = np.sqrt(0.5 - 0.5 * cos_signal[0])
            # use rough estimate to determine which set of signs to look at
            avals = list(map(float, avals))

            # find first element in avals that is larger than a0
            left = 0
            right = len(avals)
            
            while left < right:
                mid = (left + right) // 2
                
                if avals[mid] <= a0:
                    left = mid + 1
                else:
                    right = mid
            L=3
            if left >= 4:
                avals_to_use = list(map(str, avals[left-L: left+L]))
            if left >= 3:
                avals_to_use = list(map(str, avals[left-3: left+L]))
            elif left>=2:
                avals_to_use = list(map(str, avals[left-2: left+L]))
            elif left>=1:
                avals_to_use = list(map(str, avals[left-1: left+L]))
            else:
                avals_to_use = list(map(str, avals[left: left+L]))

            if disp:
                print(f'rough estimate a: {a0}')
                print(f'avals to use: {avals_to_use}')

            all_signs = []
            for a in avals_to_use:
                for x in signs_to_try[a]:
                    hamming_distance_two_signs = generate_adjacent_sign_variations(np.array(x))
                    all_signs.extend(hamming_distance_two_signs)
            all_signs = list(set(all_signs))
            if disp:
                print(f'number of signs Hamming distance two: {len(all_signs)}')

            for x in all_signs:
                curr_obj = objective_function(np.array(x), cos_signal, abs_sin, ula_signal, esprit)
                if curr_obj<obj:
                    if disp:
                        print(f'current objective: {curr_obj}')
                        print(f'current best signs: {x}')
                    obj = curr_obj
                    x_star = np.array(x)


        else:
            all_signs = []
            for a in avals:
                all_signs.extend(heavy_signs[a])
            all_signs = list(set(all_signs))
            for x in all_signs:
                basin_obj = objective_function(np.array(x), cos_signal, abs_sin, ula_signal, esprit)
                if basin_obj < obj:
                    obj = basin_obj
                    x_star = np.array(x)
    else:
        avals = list(heavy_signs.keys())
        signs_to_try = {}
        for a in avals:
            signs_to_try[a] = heavy_signs[a]

        if optimize:
            # rough estimate where the amplitude is
            a0 = np.sqrt(0.5 - 0.5 * cos_signal[0])
            # use rough estimate to determine which set of signs to look at
            avals = list(map(float, avals))

            # find first element in avals that is larger than a0
            left = 0
            right = len(avals)
            
            while left < right:
                mid = (left + right) // 2
                
                if avals[mid] <= a0:
                    left = mid + 1
                else:
                    right = mid
            L=2
            if left >= 4:
                avals_to_use = list(map(str, avals[left-L: left+L]))
            if left >= 3:
                avals_to_use = list(map(str, avals[left-3: left+L]))
            elif left>=2:
                avals_to_use = list(map(str, avals[left-2: left+L]))
            elif left>=1:
                avals_to_use = list(map(str, avals[left-1: left+L]))
            else:
                avals_to_use = list(map(str, avals[left: left+L]))

            if disp:
                print(f'rough estimate a: {a0}')
                print(f'avals to use: {avals_to_use}')

            all_signs = []
            for a in avals_to_use:
                for x in signs_to_try[a]:
                    hamming_distance_two_signs = generate_adjacent_sign_variations(np.array(x))
                    all_signs.extend(hamming_distance_two_signs)
            all_signs = list(set(all_signs))
            if disp:
                print(f'number of signs Hamming distance two: {len(all_signs)}')

            for x in all_signs:
                curr_obj = objective_function(np.array(x), cos_signal, abs_sin, ula_signal, esprit)
                if curr_obj<obj:
                    if disp:
                        print(f'current objective: {curr_obj}')
                        print(f'current best signs: {x}')
                    obj = curr_obj
                    x_star = np.array(x)


        else:
            all_signs = []
            for a in avals:
                all_signs.extend(heavy_signs[a])
            all_signs = list(set(all_signs))
            if disp:
                print(f'number of signs to try: {len(all_signs)}')
            for x in all_signs:
                basin_obj = objective_function(np.array(x), cos_signal, abs_sin, ula_signal, esprit)
                if basin_obj < obj:
                    obj = basin_obj
                    x_star = np.array(x)
    if disp:
        print(x_star)
        
    signal = cos_signal + 1.0j * x_star * abs_sin
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    theta_est, _ = esprit.estimate_theta_toeplitz(R)
    eigs = np.abs(esprit.eigs)[:2]
    basin_obj = eigs[1] - eigs[0]
    if correction:
        theta_est = apply_correction(ula_signal, measurements, theta_est, theta) #apply correction
    if disp:
        print(f'basin obj: {basin_obj}')

    cR = ula_signal.get_cov_matrix_toeplitz(csignal)
    theta_est1, _ = esprit.estimate_theta_toeplitz(cR)
    eigs = np.abs(esprit.eigs)[:2]
    true_obj = eigs[1] - eigs[0]
    if disp:
        print(f'true obj: {true_obj}')

    num_queries = np.sum(np.array(ula_signal.depths)*np.array(ula_signal.n_samples)) + ula_signal.n_samples[0]
    # Compute the maximum single query
    max_single_query = np.max(ula_signal.depths)

    ret_dict = {'theta_est': theta_est, 'theta_est1': theta_est1, 
                'error': np.abs(np.sin(theta)-np.sin(theta_est)), 
                'error1': np.abs(np.sin(theta)-np.sin(theta_est1)), 
                'queries': num_queries, 'depth': max_single_query,
                'true_obj': true_obj, 'basin_obj': basin_obj,
                'x_star': x_star}
    
    return ret_dict

csae_with_local_minimization(theta, ula_signal, esprit, heavy_signs, sample=True, correction=True, optimize=True, disp=True)

0.2
correct signs: [ 1.  1.  1. -1.  1.  1.  1.  1.  1.]
rough estimate a: 0.25398969689445183
avals to use: ['0.20566184157877418', '0.22199184853142423', '0.23825982055751374', '0.2544612116232283', '0.2705914943005945', '0.286646161032655']
number of signs Hamming distance two: 78
current objective: -215.11216039028844
current best signs: (1.0, 1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0)
current objective: -277.658017897294
current best signs: (1.0, 1.0, 1.0, -1.0, 1.0, 1.0, -1.0, -1.0, -1.0)
current objective: -281.63154299339334
current best signs: (1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, -1.0, -1.0)
current objective: -379.30786781480293
current best signs: (1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0, 1.0, 1.0)
[ 1.  1.  1. -1.  1.  1.  1.  1.  1.]
basin obj: -379.3078678148037
true obj: -379.3078678148032


{'theta_est': 0.2004110686402131,
 'theta_est1': 0.2004110686402131,
 'error': 0.0009278113661293497,
 'error1': 0.0009278113661293497,
 'queries': 2600,
 'depth': 128,
 'true_obj': -379.3078678148032,
 'basin_obj': -379.3078678148037,
 'x_star': array([ 1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.])}

## Let's do some statistics now

In [13]:
np.random.seed(42)

avals = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# avals = [0.2]
# avals = [0.5, 0.6, 0.7, 0.8, 0.9]
narray = [2]*6
heavy_signs = get_heavy_signs(narray, len(narray))
num_queries = np.zeros(len(avals), dtype=int)
max_single_query = np.zeros(len(avals), dtype=int)

num_mc=100
thetas = np.zeros((len(avals), num_mc))
errors = np.zeros((len(avals), num_mc))
thetas1 = np.zeros((len(avals), num_mc))
errors1 = np.zeros((len(avals), num_mc))

basin_obj = np.zeros((len(avals), num_mc))
true_obj = np.zeros((len(avals), num_mc))

ula_signal = TwoqULASignal(M=narray, C=3)
esprit = ESPIRIT()

for j,a in enumerate(avals):
    theta = np.arcsin(a)
    print(f'theta = {theta}')
    for i in tqdm(range(num_mc)):
        res = csae_with_local_minimization(theta, ula_signal, esprit, heavy_signs, sample=False, optimize=False, disp=False)
        thetas[j][i] = res['theta_est']
        errors[j][i] = res['error']

        thetas1[j][i] = res['theta_est1']
        errors1[j][i] = res['error1']

        basin_obj[j][i] = res['basin_obj']
        true_obj[j][i] = res['true_obj']
    num_queries[j] = res['queries']
    max_single_query[j] = res['depth']


    print(f'constant factors query and depth: {np.percentile(errors[j], 95) * num_queries[j]}, {np.percentile(errors[j], 95) * max_single_query[j]}')




theta = 0.1001674211615598


100%|██████████| 100/100 [00:04<00:00, 20.01it/s]


constant factors query and depth: 5.153695491652375, 0.41024441724596017
theta = 0.2013579207903308


100%|██████████| 100/100 [00:04<00:00, 20.83it/s]


constant factors query and depth: 254.1935620168903, 20.234313394379328
theta = 0.30469265401539747


...

constant factors query and depth: 120.56505786781861, 9.59721853674178
theta = 0.9272952180016123


100%|██████████| 100/100 [00:04<00:00, 20.89it/s]


constant factors query and depth: 1.8277744461952063, 0.1454944832792204
theta = 1.1197695149986342


100%|██████████| 100/100 [00:04<00:00, 20.85it/s]

constant factors query and depth: 99.78142913657155, 7.9428003292793266


In [14]:
for i in range(len(avals)):
    print(np.sum(np.where(np.array(basin_obj[i])/np.array(true_obj[i]) >= 0.99, 1, 0))/len(basin_obj[i]))

0.69
0.18
1.0
0.88
0.8
1.0
0.45
1.0
0.13


In [38]:
np.random.seed(42)

avals = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# avals = [np.random.uniform(0.1, 0.9)]
# avals = [0.5, 0.6, 0.7, 0.8, 0.9]
avals = [0.1, 0.2, 0.3]
avals = [np.random.uniform(0.5, 0.9) for _ in range(3)]
# narray = [2]*5 + [3]
narray = [2]*6

print('learning the distribution of signs...')
heavy_signs = get_heavy_signs(narray, int(4*len(narray)))
num_queries = np.zeros(len(avals), dtype=int)
max_single_query = np.zeros(len(avals), dtype=int)

num_mc=100
thetas = np.zeros((len(avals), num_mc))
errors = np.zeros((len(avals), num_mc))
thetas1 = np.zeros((len(avals), num_mc))
errors1 = np.zeros((len(avals), num_mc))

basin_obj = np.zeros((len(avals), num_mc))
true_obj = np.zeros((len(avals), num_mc))

ula_signal = TwoqULASignal(M=narray, C=5)
esprit = ESPIRIT()

for j,a in enumerate(avals):
    theta = np.arcsin(a)
    print(f'theta = {theta}')
    for i in tqdm(range(num_mc)):
        res = csae_with_local_minimization(theta, ula_signal, esprit, heavy_signs, sample=True, correction=True, optimize=True, disp=False)
        thetas[j][i] = res['theta_est']
        errors[j][i] = res['error']

        thetas1[j][i] = res['theta_est1']
        errors1[j][i] = res['error1']

        basin_obj[j][i] = res['basin_obj']
        true_obj[j][i] = res['true_obj']
    num_queries[j] = res['queries']
    max_single_query[j] = res['depth']

    print(f'constant factors query and depth: {np.percentile(errors[j], 95) * num_queries[j]}, {np.percentile(errors[j], 95) * max_single_query[j]}')

learning the distribution of signs...
theta = 0.7073423981098835


100%|██████████| 100/100 [00:09<00:00, 10.13it/s]


constant factors query and depth: 5.047470168911924, 0.24107320209728592
theta = 1.076464090602309


100%|██████████| 100/100 [00:10<00:00,  9.74it/s]


constant factors query and depth: 1.6628874003715497, 0.07942148777893969
theta = 0.9153854581288509


100%|██████████| 100/100 [00:10<00:00,  9.80it/s]

constant factors query and depth: 3.503333415480857, 0.16732338700804092


In [40]:
for i in range(len(avals)):
    print(np.sum(np.where(np.array(basin_obj[i])/np.array(true_obj[i]) >= 0.99, 1, 0))/len(basin_obj[i]))

0.98
0.99
1.0


In [30]:
i=2
np.array(basin_obj[i])/np.array(true_obj[i]), np.sum(np.where(np.array(basin_obj[i])/np.array(true_obj[i]) >= 0.99, 1, 0))/len(basin_obj[i])

(array([1.04622707, 1.        , 1.        , 1.04172664, 1.00054807,
        1.        , 1.07144225, 1.00158999, 1.03053211, 1.05616181,
        1.00883563, 1.00019169, 1.00399656, 1.01475438, 1.00622753,
        1.01995641, 1.        , 1.        , 1.01304515, 1.        ,
        1.        , 1.02735191, 1.        , 1.        , 1.        ,
        1.0543875 , 1.        , 1.01950676, 1.        , 1.        ,
        1.0051716 , 1.00998773, 1.        , 1.        , 1.01300575,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.00528447, 1.14180628, 1.        ,
        1.00084852, 1.00295001, 1.        , 1.        , 1.3899624 ,
        1.00364448, 1.        , 1.        , 1.00167909, 1.0227291 ,
        1.00300997, 1.00057497, 0.99153967, 1.        , 1.12532705,
        1.        , 1.00326244, 1.1203493 , 1.        , 0.9992473 ,
        1.        , 1.        , 1.        , 1.00670482, 1.        ,
        1.01845463, 1.01379585, 1.00125124, 1.  

In [31]:
np.sin(thetas[i]), avals[i]

(array([ 0.34100766,  0.3219452 ,  0.31903149,  0.32361739,  0.31817794,
         0.31979209,  0.3158515 ,  0.31953527,  0.31538166,  0.32443249,
         0.3160303 ,  0.32648398,  0.31979343,  0.34017161,  0.31944942,
         0.31857608,  0.31806788,  0.32177377,  0.33944209,  0.31440854,
         0.32583992,  0.3197517 ,  0.31724326,  0.32173441,  0.31915967,
         0.32430029,  0.31912938,  0.31841071,  0.31601767,  0.31745404,
         0.32135682,  0.31599551,  0.31669707,  0.32227794,  0.32286096,
         0.32251445,  0.31800379,  0.3185536 ,  0.32139924,  0.31850476,
         0.32160444,  0.31733349,  0.32132075,  0.32168481,  0.32369924,
         0.32165546,  0.3219062 ,  0.31942124,  0.32454267,  0.3165256 ,
         0.31555047,  0.31626846,  0.32175548,  0.3253274 ,  0.32253585,
         0.32120748, -0.32434622,  0.31869608,  0.31641876,  0.3137601 ,
         0.32437862,  0.32517706,  0.32233451,  0.31835712,  0.31745301,
         0.31467255,  0.31592886,  0.31365387,  0.3

In [41]:
perc = 68
avg = 0.0
for i in range(len(avals)):
    avg += np.percentile(errors[i], perc) * num_queries[i]/len(avals)
    print(f'constant factors query and depth ({perc}%) for {avals[i]}: {np.percentile(errors[i], perc) * num_queries[i]}, {np.percentile(errors[i], perc) * max_single_query[i]}')
avg

constant factors query and depth (68%) for 0.649816047538945: 1.7090592980363317, 0.08162671274203376
constant factors query and depth (68%) for 0.8802857225639664: 0.9795226274791214, 0.04678317026765953
constant factors query and depth (68%) for 0.7927975767245621: 1.3103674067705313, 0.0625847119651597


1.332983110761995

In [20]:
Q = 7
lengths = []
for q in range(2, Q + 1):
    narray = [2]*(2*q)
    heavy_signs = get_heavy_signs(narray, 4*len(narray))
    signs_to_try = []
    for a in heavy_signs.keys():
        signs = heavy_signs[a].keys()
        signs_to_try += signs
    signs_to_try = list(set(signs_to_try))
    lengths.append(len(signs_to_try))

lengths

[20, 57, 106, 208, 330, 461]